# Notebook for map art
### Ideas:
- make proper 3D map 
    - with persepective (develop based on shadow projection alogrithm)
    - with light 
        - based on orientation or 
        - with realistic light including shadows
    - use glass like optic
- use shadow projection to give the map itself some thickness or use the overlay
    - how to deal with form intersection the edge? maybe do something like a waterfall (plot hull in color, and after some depth stop this, so that lines keep falling down (actually reverse as plot from bottom to top))
- transform all polygons as if they would fall down (select most downwards point and turn polygon around it)

### 3D map by overlaying layers and permuting each time.
- has perspective but parallel, like beeing infinitely far away and having zoomed in 

In [ ]:
# some map art

from shapely.affinity import translate
from shapely.ops import unary_union
import matplotlib.pyplot as plt

sf = ["Prenzlauer Berg"]#["Pankow", "Prenzlauer Berg", "Mitte"]
spb = buildings[buildings["zone"].isin(sm.zones.index[sm.zones["nam"].isin(sf)])]
bspb = unary_union(list(spb.geometry.apply(lambda g: g.buffer(0.002))))
bspb = Polygon(bspb.exterior.coords)

fig, ax = plt.subplots(figsize = (16,16))

maxHeight = max(spb["ew_ha_2020"])

nSteps = 50
cbuff = 40
cmap = plt.get_cmap('Reds_r', nSteps + cbuff)

for i in range(0,nSteps):
    # spb["geometry"] = spb.apply(lambda row : translate(row["geometry"], 1,8) if (row["ew_ha_2020"] >= i/nSteps*maxHeight) else None, axis = 1)
    spb["geometry"] = spb.apply(lambda row : translate(row["geometry"], 0.00001,0.00008) if (row["ew_ha_2020"] >= i/nSteps*maxHeight) else None, axis = 1)
    spb.plot(ax=ax, color = cmap(round(i + 0.5*cbuff)), edgecolor = cmap(round(i + 0.5*cbuff)), linewidth = 2)
    # spb.plot(ax=ax, column='ew_ha_2020', cmap = "viridis", linewidth = 2)
    




### Segement Line to plot line in color gradiant
- smoothness could improved by overlay and opacity < 1
- could be used to indicate trajectory/ path. 
  

In [ ]:
from shapely.geometry import LineString, Point
# helper function to divide a line into segments from start to end point
def segmentLine(l=LineString([(0,0),(1,1)]), i=0, nSteps=10):
    startPoint = Point(
        i/nSteps*l.coords[0][0] + (1-(i/nSteps)) * l.coords[-1][0],
        i/nSteps*l.coords[0][1] + (1-(i/nSteps)) * l.coords[-1][1]
    )
    i += 1
    endPoint = Point(
        i/nSteps*l.coords[0][0] + (1-(i/nSteps)) * l.coords[-1][0],
        i/nSteps*l.coords[0][1] + (1-(i/nSteps)) * l.coords[-1][1]
    )
    return LineString([startPoint, endPoint])


In [ ]:
# show zones and popCentroids and their offsets from real geometrical centroids
fig, ax = plt.subplots(1, 1, figsize=(16,16))
buildings.plot(ax=ax, column='ew_ha_2020', cmap = "Reds")
# add zone borders and a bit of opacity
sm.zones.geometry.plot(ax=ax, color = "#ffffff33", edgecolor="#999999")
nSteps = 20
cbuff = 10
cmap = plt.get_cmap('Greens_r', nSteps + cbuff)
gpd.GeoSeries(sm.zones.geometry.centroid, crs=sm.epsg).plot(ax=ax, color = cmap(nSteps), markersize = 20)
for i in range(1,nSteps):
    l = [helpers.segmentLine(x, i, nSteps) for x in sm.zones.popCentroidOffset]
    gpd.GeoSeries(l, crs=sm.epsg).plot(ax=ax,color=cmap(i), linewidth=3)
gpd.GeoSeries(sm.zones.popCentroid, crs=sm.epsg).plot(ax=ax, color = cmap(0), markersize = 20)
ax.set_title("Zones centroids vs population weigthed centroids")